Candidate details:

Name: Natraj Vikram

email: vikram.iisccamp@gmail.com

### Problem Statement

Given: An XLS with the following columns:-

- line_item_name: this is the raw line item name extracted from the invoice
- line_item_description: this is the raw line item description extracted from the invoice
- canonical_vendor_name: this is the canonical vendor name to which the invoice has been mapped
- canonical_line_item_name: this is the canonical line item name to which the raw line item name has been mapped / will be mapped


### Task and objective:
- Map raw line item names (with descriptions) on invoices to canonical line itemnames. 

This is a common task in NLP known as entity linking.

### Approach used:

- The dataset given is labeled.

- Since the amount of data given to us is very limited , we can't expect good performance if we train a model from scratch. The resulting output may not be good because the corpus size provided is not enough for the model to train and generalize well.

- Therefore, we train a custom NER model by fine tuning some existing pretrained language models from spaCy. 

- For this,I've fine tune 3 language models from spacy namely :- en_core_web_sm,en_core_web_md and en_core_web_lg.


### Adding custom labels to the existing spacy nlp model:

- The spaCy library offers pretrained entity extractors.(Person,Organization,Country etc.)

- But for custom business case use requirements,in financial and marketing domains,these pre-existing entities may not be enough. Therefore,we add a new entity called "CLI",short for canonical line item and provide that in the training data.
- After training,We will try to find out what parts of text belong to the entity type CLI using the test/eval data.

In [1]:
#Import all the necessary libraries
import pandas as pd
from tqdm import tqdm,trange
import numpy as np
import re
import spacy
import random

In [2]:
# Load pre-existing spacy language model with english corpus

# nlp=spacy.load('en_core_web_sm')
# nlp=spacy.load('en_core_web_md')
nlp=spacy.load('en_core_web_lg')


In [3]:
nlp.pipe_names
#predefined pipe names already present in the model

['tagger', 'parser', 'ner']

In [4]:
# Getting the pipeline component
ner=nlp.get_pipe("ner")

In [5]:
df = pd.read_excel('Task.xlsx',sheet_name='train')
df = df.fillna('')
df['line_item_name'] = df['line_item_name'].astype(str) 
df['line_item_description'] = df['line_item_description'].astype(str) 


In [6]:
#Since we have to map raw line item names (with descriptions) on invoices to canonical line itemnames,
#we create a new column name_desc which will combine item name with description
df['name_desc'] = np.zeros(len(df))
for i in trange(len(df)):
    df['name_desc'].iloc[i] = df['line_item_name'].iloc[i] + ' ' + df['line_item_description'].iloc[i]

  0%|          | 0/659 [00:00<?, ?it/s]/home/vikram/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
100%|██████████| 659/659 [00:00<00:00, 8497.93it/s]


### Prepare train data
- As per the spaCy documentation,the train data should be present in the following format.
- The format of the training data is a list of tuples. Each tuple contains the example text and a dictionary. The dictionary will have the key entities , that stores the start and end indices along with the label of the entitties present in the text.

- For example , To pass “Pizza is a common fast food” as example the format will be : 
("Pizza is a common fast food",{"entities" : [(0, 5, "FOOD")]})


In [7]:
def find_start_end(string,substring):
    start = string.index(substring)
    end = string.index(substring) + len(substring) 
    return start,end  ## this returns start and end indices of a substring

In [8]:
ix = []
ent = []
ent_ix = []
for i in trange(len(df)):
    try:
        
        ent.append([find_start_end(df['name_desc'].iloc[i],df['canonical_line_item_name'].iloc[i])+("CLI",)])
        ent_ix.append(i)
        
    except:
        ent.append(0)
        ix.append(i)

100%|██████████| 659/659 [00:00<00:00, 59172.08it/s]


In [9]:
df['entities'] = ent

In [10]:
df = df[df.entities != 0]

In [11]:
df = df[['line_item_name', 'line_item_description', 'canonical_vendor_name',
       'name_desc', 'canonical_line_item_name','entities']]

In [12]:
df

,line_item_name,line_item_description,canonical_vendor_name,name_desc,canonical_line_item_name,entities
0,Management Services,April 2019 Services,10 Minute Ventures,Management Services April 2019 Services,Management Services,"[(0, 19, CLI)]"
1,June Web Media Fee,"($249,300 x 12% Commission)",Acqcom Digital Marketing,"June Web Media Fee ($249,300 x 12% Commission)",Web Media Fee,"[(5, 18, CLI)]"
2,June Web Media Fee,"($180,000 x 12% Commission)",Acqcom Digital Marketing,"June Web Media Fee ($180,000 x 12% Commission)",Web Media Fee,"[(5, 18, CLI)]"
3,Business Package,,Adjust,Business Package,Business Package,"[(0, 16, CLI)]"
4,SEO Services,,AdLift,SEO Services,SEO Services,"[(0, 12, CLI)]"
...,...,...,...,...,...,...
545,Domain Privacy Registration Fees via $261.70,,Waltz Media Group,Domain Privacy Registration Fees via $261.70,Domain Privacy Registration Fees,"[(0, 32, CLI)]"
547,"Business Hosting (Year, USD) (at $432.00 / yea...",,Webflow,"Business Hosting (Year, USD) (at $432.00 / yea...",Business Hosting,"[(0, 16, CLI)]"
598,Service address charge x4,Service address charge x4,Wilson Stevens,Service address charge x4 Service address char...,Service address charge,"[(0, 22, CLI)]"
604,Workable Annual Plan - Platform fee,,Workable,Workable Annual Plan - Platform fee,Workable Annual Plan - Platform fee,"[(0, 35, CLI)]"


In [13]:
TRAIN_DATA = []
for i in range(len(df)):
    TRAIN_DATA.append((df['name_desc'].iloc[i],{'entities':df['entities'].iloc[i]}))

In [14]:
TRAIN_DATA[0:5]
#first 5 points from the train data

[('Management Services April 2019 Services', {'entities': [(0, 19, 'CLI')]}),
 ('June Web Media Fee ($249,300 x 12% Commission)',
  {'entities': [(5, 18, 'CLI')]}),
 ('June Web Media Fee ($180,000 x 12% Commission)',
  {'entities': [(5, 18, 'CLI')]}),
 ('Business Package ', {'entities': [(0, 16, 'CLI')]}),
 ('SEO Services ', {'entities': [(0, 12, 'CLI')]})]

In [15]:
#Adding labels to the `ner`
# Adding custom label CLI - canonical line item
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

In [15]:
# ner.add_label("CLI")
# assert "CLI" in ner.labels

In [16]:
ner.labels
#here we see that the CLI has been added. Just a sanity check.

('CARDINAL',
 'CLI',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')

In [17]:
ner.labels[1]

'CLI'

In [18]:
# Disable pipeline components you dont need to change
# pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
pipe_exceptions = ["ner"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [19]:
unaffected_pipes

['tagger', 'parser']

### Train model

In [20]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):
    
    optimizer = nlp.resume_training()

  # Training for 30 iterations
    for iteration in trange(30):
        

    # shuufling examples  before every iteration
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                        texts,  # batch of texts
                        annotations,  # batch of annotations
                        drop=0.5,  # dropout - make it harder to memorise data
                        losses=losses,
                        sgd = optimizer
                    )
            print("Losses", losses)

  0%|          | 0/30 [00:00<?, ?it/s]

Losses {'ner': 66.65388107299805}
Losses {'ner': 150.91592979431152}
Losses {'ner': 196.2850785255432}
Losses {'ner': 253.73339128494263}
Losses {'ner': 309.7413754463196}
Losses {'ner': 387.25587224960327}
Losses {'ner': 453.43138456344604}
Losses {'ner': 571.7008635997772}
Losses {'ner': 597.9463069438934}
Losses {'ner': 617.8466880321503}
Losses {'ner': 733.9416263103485}
Losses {'ner': 775.9578865766525}
Losses {'ner': 828.2640651464462}
Losses {'ner': 877.222648024559}
Losses {'ner': 914.7626446485519}
Losses {'ner': 945.2591818571091}
Losses {'ner': 981.9784973859787}
Losses {'ner': 1047.838891863823}
Losses {'ner': 1119.6160868406296}
Losses {'ner': 1171.1442540884018}
Losses {'ner': 1207.6202725172043}
Losses {'ner': 1230.5415757894516}
Losses {'ner': 1288.4488800764084}
Losses {'ner': 1485.6891325712204}
Losses {'ner': 1531.9893645048141}
Losses {'ner': 1598.8106735944748}
Losses {'ner': 1634.714642882347}
Losses {'ner': 1662.8551174402237}
Losses {'ner': 1713.4850422143936}
L

/home/vikram/anaconda3/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "1 Verifly_CPC - SmallBusinessInsurance: SureHits M..." with entities "[(16, 82, 'CLI')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)


Losses {'ner': 2303.08339035511}
Losses {'ner': 2339.5543640851974}
Losses {'ner': 2385.0630229711533}
Losses {'ner': 2421.089947581291}
Losses {'ner': 2471.572889685631}
Losses {'ner': 2526.786007285118}
Losses {'ner': 2629.4723876714706}
Losses {'ner': 2659.3899418115616}
Losses {'ner': 2706.067607522011}
Losses {'ner': 2726.164668083191}
Losses {'ner': 2776.4133501052856}
Losses {'ner': 2824.0950179100037}
Losses {'ner': 2911.9591059684753}
Losses {'ner': 2966.032901287079}
Losses {'ner': 3041.515498161316}
Losses {'ner': 3101.9174480438232}
Losses {'ner': 3196.8952226638794}
Losses {'ner': 3255.536959886551}
Losses {'ner': 3311.2325117588043}
Losses {'ner': 3325.8400418758392}
Losses {'ner': 3377.094755411148}
Losses {'ner': 3457.766608476639}
Losses {'ner': 3514.2773110866547}
Losses {'ner': 3575.86719250679}
Losses {'ner': 3617.903891801834}
Losses {'ner': 3687.084585428238}
Losses {'ner': 3711.1476607322693}
Losses {'ner': 3755.176158428192}
Losses {'ner': 3813.024025440216}
Los

/home/vikram/anaconda3/lib/python3.7/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Accounting:Core_Accounting_Service Unbilled Hours-..." with entities "[(35, 49, 'CLI')]". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)
  3%|▎         | 1/30 [00:02<01:04,  2.21s/it]

...

Losses {'ner': 35.28943920135498}
Losses {'ner': 71.07767653465271}
Losses {'ner': 100.95375561714172}
Losses {'ner': 121.05743551254272}
Losses {'ner': 155.36130475997925}
Losses {'ner': 190.82702159881592}
Losses {'ner': 234.85257387161255}
Losses {'ner': 266.5381543636322}
Losses {'ner': 284.5590073466301}
Losses {'ner': 316.96600979566574}
Losses {'ner': 358.6381451487541}
Losses {'ner': 426.7458578944206}
Losses {'ner': 445.4650943875313}
Losses {'ner': 506.63967686891556}
Losses {'ner': 547.6566050648689}
Losses {'ner': 576.5760787129402}
Losses {'ner': 628.9701860547066}
Losses {'ner': 654.8783071637154}
Losses {'ner': 686.8313749432564}
Losses {'ner': 719.6971605420113}
Losses {'ner': 754.0935462117195}
Losses {'ner': 765.6820469498634}
Losses {'ner': 794.7797678112984}
Losses {'ner': 816.1762242913246}
Losses {'ner': 846.5466161370277}
Losses {'ner': 873.5193579792976}
Losses {'ner': 895.7432926297188}
Losses {'ner': 921.4738066792488}
Losses {'ner': 957.4160278439522}
Losses 

 97%|█████████▋| 29/30 [00:59<00:02,  2.26s/it]

Losses {'ner': 3225.2156196832657}
Losses {'ner': 3269.5864320993423}
Losses {'ner': 3275.028653898298}
Losses {'ner': 22.42706561088562}
Losses {'ner': 55.790393590927124}
Losses {'ner': 117.97295212745667}
Losses {'ner': 145.53264784812927}
Losses {'ner': 183.33447790145874}
Losses {'ner': 224.95560312271118}
Losses {'ner': 253.74021935462952}
Losses {'ner': 275.9021580219269}
Losses {'ner': 321.6686751842499}
Losses {'ner': 345.09603452682495}
Losses {'ner': 364.56579756736755}
Losses {'ner': 428.48092770576477}
Losses {'ner': 445.14167380332947}
Losses {'ner': 484.9329068660736}
Losses {'ner': 527.0433223247528}
Losses {'ner': 553.6901714801788}
Losses {'ner': 602.5732724666595}
Losses {'ner': 629.7142231464386}
Losses {'ner': 645.2917490005493}
Losses {'ner': 685.1467385292053}
Losses {'ner': 712.7895369529724}
Losses {'ner': 734.3780164718628}
Losses {'ner': 764.9751133918762}
Losses {'ner': 811.2402873039246}
Losses {'ner': 847.6402697563171}
Losses {'ner': 920.5125842094421}
Lo

100%|██████████| 30/30 [01:01<00:00,  2.07s/it]

Losses {'ner': 2947.815644264221}
Losses {'ner': 2999.6393508911133}
Losses {'ner': 3045.1209292411804}
Losses {'ner': 3070.401665687561}
Losses {'ner': 3098.9978079795837}
Losses {'ner': 3135.0347352027893}
Losses {'ner': 3203.44092464447}
Losses {'ner': 3210.0688272938132}


In [21]:
nlp.to_disk("./model_lg")
#save model

### Test predictions of unseen data using the trained model

In [1]:
####load trained ner spacy model

In [1]:
import pandas as pd
from tqdm import tqdm,trange
import numpy as np
import re
import spacy
import random

In [2]:
output_dir = './model_lg'
print("Loading from", output_dir)
nlp = spacy.load(output_dir)

Loading from ./model_lg


In [ ]:
### Find out the predicted entities from the eval tab

In [3]:
df = pd.read_excel('Task.xlsx',sheet_name='eval')
df = df.fillna('')
df['line_item_name'] = df['line_item_name'].astype(str) 
df['line_item_description'] = df['line_item_description'].astype(str) 
df['name_desc'] = np.zeros(len(df))
for i in trange(len(df)):
    df['name_desc'].iloc[i] = df['line_item_name'].iloc[i] + ' ' + df['line_item_description'].iloc[i]
for i in range(len(df)):
    
    df['name_desc'].iloc[i] = " ".join(df['name_desc'].iloc[i].split())

  0%|          | 0/337 [00:00<?, ?it/s]/home/vikram/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
100%|██████████| 337/337 [00:00<00:00, 8898.15it/s]


In [4]:
df = df[['line_item_name', 'line_item_description', 'canonical_vendor_name',
       'name_desc', 'canonical_line_item_name']]

In [5]:
# Testing the model
predicted_entities = []
for i in trange(len(df)):
    doc = nlp(df['name_desc'].iloc[i])
    output = [(ent.text, ent.label_) for ent in doc.ents]
    print("Entities:",output)
    try:
        
        predicted_entities.append(output[0][0])  ##check if the model is predicting
    except:
        predicted_entities.append('')  ##append empty string when there are no predictions

  8%|▊         | 28/337 [00:00<00:02, 132.53it/s]

Entities: [('Management Services May', 'CLI')]
Entities: [('Acrobat Pro DC', 'CLI')]
Entities: [('AIEX 96 Pieces', 'CLI')]
Entities: [('AmazonBasics AAA', 'CLI')]
Entities: [('AmazonBasics Mesh Trash Can Waste Basket 1', 'CLI')]
Entities: [('AmazonFresh Mediterranean Extra Virgin Olive Oil, 68 Fl Oz (2L) B01N3LCEDL', 'CLI')]
Entities: [('Apple 87W USB-C Power Adapter (for MacBook Pro)', 'CLI')]
Entities: [('Apple iPad 2', 'CLI')]
Entities: [('Apple iPad with Retina Display', 'CLI')]
Entities: [('Apple iPad with Retina Display', 'CLI')]
Entities: [('Assurant B2B 4YR Kitchen Protection Plan with Accidental Damage $50-74', 'CLI')]
Entities: [('Bose QuietComfort 35 (Series I) Wireless Headphones,', 'CLI')]
Entities: [('Calculator,Vilcome 12-Digit Solar Battery Office Calculator with Large LCD Display Big Sensitive Button, Dual Power Desktop Calculators (Black) Calculator,Vilcome 12-Digit Solar Battery Office Calculator with Large LCD Display Big Sensitive Button, Dual Power Desktop Calcula

 17%|█▋        | 58/337 [00:00<00:01, 139.78it/s]

Entities: [('Tilting TV Wall Mount Bracket Low Profile for Most 23-55 Inch LED, LCD, OLED, Plasma Flat Screen TVs with VESA 400x400mm Weight up to 115lbs by PERLES', 'CLI')]
Entities: [('TRENDnet USB-C HD Docking Cube, HDMI, Gigabit Port, USB Docking Station, Supports up to 3840 x 2160 at 30 Hz, TUC-DS1', 'CLI')]
Entities: [('Whaline 120 Pack Halloween Gift Tags Trick or Treat Craft Label Tags with 130ft Cotton Rope for Halloween Party Favor, Candy Goodie Bag Art', 'CLI')]
Entities: [('Xbox Wireless Controller - Black', 'CLI')]
Entities: [('E-mail gift card to: christopher.byrum@thimble.com', 'CLI')]
Entities: []
Entities: [('Lenovo 500 Wireless Combo Keyboard & Mouse, Black (GX30H55793)', 'CLI')]
Entities: []
Entities: [('1,500-2,000 words', 'CLI')]
Entities: [('1,500-2,000 words', 'CLI')]
Entities: [('900-1,500 words', 'CLI')]
Entities: [('900-1,500 words @', 'CLI')]
Entities: [('900-1,500 words @', 'CLI')]
Entities: [('Trial Assignment @ $350 Miscellaneous Professional Liability Ins

 27%|██▋       | 92/337 [00:00<00:01, 152.69it/s]

...

Entities: [('Staples Carpet Chair Mat, 36" x 48\'\', Crystal Clear (STP-17436) Staples Carpet Chair Mat, 36" x 48\'\', Crystal Clear (STP-17436)', 'CLI')]
Entities: [('Internet Access Internet Access : New York - Chinatown / LES - Stealth Fiber 1', 'CLI')]
Entities: []
Entities: [('m Limited Fixed Cost: Barracuda Sentinel BSENTS001a:', 'CLI'), ('Barracuda Sentinel for Office 365 - subscription license (1 year):', 'CLI')]
Entities: [('m Limited Fixed Cost: Barracuda Sentinel', 'CLI')]
Entities: [('jabra:', 'CLI')]
Entities: [('Billable Expenses Computer Accessories', 'CLI')]
Entities: [('Engineer Eric Gross', 'CLI')]
Entities: [('Engineer Michael Kerr', 'CLI')]
Entities: []
Entities: [('Billable Services Engineer', 'CLI')]
Entities: [('Engineer Alex Lora', 'CLI')]
Entities: [('Engineer Revor Vicencio', 'CLI')]
Entities: [('Engineer Ajoykumar Rajasekaran', 'CLI')]
Entities: []
Entities: [('Leads 43952', 'CLI')]
Entities: [('Junk hauling', 'CLI')]
Entities: [('Scotch 11.25" x 8.75', 'CLI'

 91%|█████████▏| 308/337 [00:02<00:00, 148.07it/s]

Entities: [('Filing Fees J. Smith license applications in 27 states.', 'CLI')]
Entities: [('Filing Fees Verifly entity renewal in Washington.', 'CLI')]
Entities: [('Filing Fees Verifly entity renewal in the following states with the accompanying state fees: Alaska ($80), California ($193), Colorado ($152), District of Columbia ($105), Idaho ($65), Illinois ($155), New Hampshire ($155), New Jersey ($155), Oregon ($155), Pennsylvania ($115), South Dakota ($5), Utah ($90), and West Virginia ($205).', 'CLI')]
Entities: [('Filing Fees Secretary of State DBA Name Registration in Vermont.', 'CLI')]
Entities: [('Lexis Fees Monthly Lexis', 'CLI')]
Entities: [('UPS Fees - EX', 'CLI')]
Entities: [('UPS Fees Monthly UPS shipment(s).', 'CLI')]
Entities: [('General Pulled New Mexico affiliation renewal form and list of affiliations.', 'CLI')]
Entities: [('General CC', 'CLI')]
Entities: [('KGL Correspondence with New York regarding DP Chnage.', 'CLI')]
Entities: [('Professional Services KGL', 'CLI')]

100%|██████████| 337/337 [00:02<00:00, 152.33it/s]

Entities: [('Arch 1.', 'CLI')]
Entities: [('Mike Data 1.', 'CLI')]
Entities: [('2 Mike Data 1.', 'CLI')]
Entities: [('Misc Rent (per seat x 17)', 'CLI')]
Entities: []
Entities: [('Misc Rent (per seat x 11)', 'CLI')]
Entities: [('Misc LCD ($425)', 'CLI')]
Entities: [('misc xiamen ->', 'CLI')]
Entities: [('Misc nyc', 'CLI')]
Entities: [('Misc Isaacâ€™s trip for visa interview (round-trip flight and one night hotel)', 'CLI')]
Entities: [('Misc Rent (per seat x 16)', 'CLI')]
Entities: [('Misc Rent (per seat x 8)', 'CLI')]
Entities: [('Misc macbook ($1,500)', 'CLI')]
Entities: []
Entities: [('Maggie PM 1.', 'CLI')]
Entities: [('Tina PM 1.', 'CLI')]
Entities: []
Entities: [('Jesse Web 1.', 'CLI')]
Entities: [('Jesse Web 1.', 'CLI')]
Entities: [('Web 1.', 'CLI')]
Entities: [('Sam Web 1.', 'CLI')]
Entities: [('Sam Web 1.', 'CLI')]
Entities: [('Sam Web 1.', 'CLI')]
Entities: [('Conducted a 10-15 minute email survey of up to 6,000 Verifly members Frequencies and cross-tabulation tables: a breakd

In [6]:
predicted_entities

['Management Services May',
 'Acrobat Pro DC',
 'AIEX 96 Pieces',
 'AmazonBasics AAA',
 'AmazonBasics Mesh Trash Can Waste Basket 1',
 'AmazonFresh Mediterranean Extra Virgin Olive Oil, 68 Fl Oz (2L) B01N3LCEDL',
 'Apple 87W USB-C Power Adapter (for MacBook Pro)',
 'Apple iPad 2',
 'Apple iPad with Retina Display',
 'Apple iPad with Retina Display',
 'Assurant B2B 4YR Kitchen Protection Plan with Accidental Damage $50-74',
 'Bose QuietComfort 35 (Series I) Wireless Headphones,',
 'Calculator,Vilcome 12-Digit Solar Battery Office Calculator with Large LCD Display Big Sensitive Button, Dual Power Desktop Calculators (Black) Calculator,Vilcome 12-Digit Solar Battery Office Calculator with Large LCD Display Big Sensitive Button, Dual Power Desktop Calculators (Black)',
 'Dell 130-WATT 3-Prong AC Adapter with 6 FT Power Cord',
 'Logitech K400 Plus Wireless Touch TV Keyboard with Easy Media Control and Built-',
 'Microsoft Natural Ergonomic Keyboard 4000 for Business - Wired',
 'Mouthwash Di

In [7]:
df['predicted_canonical_line_item_name'] = predicted_entities

In [8]:
df = df.drop(['canonical_line_item_name'],axis=1)

In [9]:
df

,line_item_name,line_item_description,canonical_vendor_name,name_desc,predicted_canonical_line_item_name
0,Management Services,May 2019 Services,10 Minute Ventures,Management Services May 2019 Services,Management Services May
1,Acrobat Pro DC,,Adobe,Acrobat Pro DC,Acrobat Pro DC
2,AIEX 96 Pieces Adhesive Poster Tacky Putty Sti...,,Amazon Business,AIEX 96 Pieces Adhesive Poster Tacky Putty Sti...,AIEX 96 Pieces
3,AmazonBasics AAA 1.5 Volt Performance Alkaline...,,Amazon Business,AmazonBasics AAA 1.5 Volt Performance Alkaline...,AmazonBasics AAA
4,AmazonBasics Mesh Trash Can Waste Basket,1,Amazon Business,AmazonBasics Mesh Trash Can Waste Basket 1,AmazonBasics Mesh Trash Can Waste Basket 1
...,...,...,...,...,...
332,Web,1. Thimble Monthly Webapp 2. Customer Referral...,Xiamen ZhiZhi Tech,Web 1. Thimble Monthly Webapp 2. Customer Refe...,Web 1.
333,Sam Web,1. Web app 2.0 mobile version 2. Purchase wid...,Xiamen ZhiZhi Tech,Sam Web 1. Web app 2.0 mobile version 2. Purch...,Sam Web 1.
334,Sam Web,1. Segment Integration 2. Iterable Integration,Xiamen ZhiZhi Tech,Sam Web 1. Segment Integration 2. Iterable Int...,Sam Web 1.
335,Sam Web,1. Broker Purchase Project 2. New Insurance Pr...,Xiamen ZhiZhi Tech,Sam Web 1. Broker Purchase Project 2. New Insu...,Sam Web 1.


In [10]:
df.to_csv('eval_predictions_lg.csv',index=False)

### Observation and Results:

- We trained and fine tuned the model using a few hundred datapoints,so the results are far from perfect. So the model has not generalized perfectly but it does a decent job. This approach gives us a good starting point to identify and map custom entities.
- The predicted entites / results obtained from 3 models are present in 3 different csv files inside folder output_eval.
- Result maybe further improved by using more data,providing annotations in a better way or changing the model's parameters while training.